# Topic Model Analysis

Todo: 1. industry : cusip to naics mapping

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

from sklearn import preprocessing

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from gensim.test.utils import datapath

from compute_topic import *

## Document Topic Distribution
Below, we will get top three topics per document with minimum probability threshold

In [2]:
import os
from pathlib import Path

# The model with model_name we want to load
def load_model(model_name,topics):
    current_path = os.getcwd()
    current_path = Path(current_path)
    topic_name ="_{}topics".format(topics)
    file_name = model_name + topic_name
    load_path = current_path.parents[0] / "./LDAModel/{}/{}".format(file_name,file_name)
    load_path = str(load_path)
    lda = gensim.models.ldamulticore.LdaMulticore.load(load_path)
    return lda

In [3]:
model_name = "matrix_1"
from gensim.corpora import Dictionary
corpus = load_corpus(model_name)
id2word = load_id2word(model_name)
dictionary = Dictionary.from_corpus(corpus,id2word=id2word)

loading corpus...
corpus successfully loaded!!
MmCorpus(98304 documents, 12078 features, 4072730 non-zero entries)
loading id2word ...
id2word loaded!!


In [4]:
# compute matrix to get for later creating data frame
matrix_1 = compute_matrix1()

loading data TRACE2014_jinming...
Getting data fromC:\Users\raymo\UMD\Research\FINRA_TRACE\Data\Pickle\TRACE2014_jinming...
Data getting success!
computing matrix_1 ......
computing matrix_1 done!


In [27]:
def document_topic_distribution(matrix_object,model_name,topics,minimum_probability=0.10):
    model = load_model(model_name,topics)
    # minimum_probability is our threshold
    document_topics = model.get_document_topics(corpus,minimum_probability=minimum_probability)
    # convert document_topics, which is a gesim corpus, to numpy array
    document_topic_distribution_numpy = gensim.matutils.corpus2dense(document_topics,num_terms=int(topics))
    # need to transpose it because gensim represents documents on columns token on index
    document_topic_distribution_numpy = np.transpose(document_topic_distribution_numpy)
    # combine document_topic_distribution with index from matrix and columns represents gensim topics
    document_topic_distribution_pandas = pd.DataFrame(data=document_topic_distribution_numpy,index=matrix_1.index,columns=np.arange(int(topics)))
    # Only get the top three topics per document
    document_topic_distribution_pandas = document_topic_distribution_pandas[document_topic_distribution_pandas.rank(axis=1,method='max',ascending=False) <= 3]
    # Save the dataframe to csv
    document_topic_distribution_pandas.to_csv('../Results/{}_{}topics.csv'.format(model_name,topics))

## Convergence Test

In [ ]:
import re
import matplotlib.pyplot as plt
def convergence_likelyhood(model_name,num_topics):
    p = re.compile("(-*\d+\.\d+) per-word .* (\d+\.\d+) perplexity")
    matches = [p.findall(l) for l in open('../LDAModel/{}_{}topics.log.txt'.format(model_name,num_topics))]
    matches = [m for m in matches if len(m) > 0]
    tuples = [t[0] for t in matches]
    perplexity = [float(t[1]) for t in tuples]
    liklihood = [float(t[0]) for t in tuples]
    iter = list(range(0,len(tuples)*10,10))
    plt.plot(iter,liklihood,c="black")
    plt.ylabel("log liklihood")
    plt.xlabel("iteration")
    plt.title("Topic Model Convergence")
    plt.grid()
    plt.savefig("{}_{}topics.pdf".format(model_name,num_topics))
    plt.close()

In [ ]:
#convergence_likelyhood("matrix_1",250)